## [MAUVE & ROUGE] ASQA & ELI5

In [ ]:
import copy
import json
from metrics import *

with open("/shared/eng/pj20/firas_data/test_datasets/results/asqa_sonnet_retrieval_top_5_base.json", "r") as f:
    data = json.load(f)['data']

In [ ]:
normalized_data = copy.deepcopy(data)
for i in range(len(normalized_data)):
    normalized_data[i]['output'] = remove_citations(normalized_data[i]['output'])

references = [' '.join((item['question'] + " " + item['answer'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]
predictions = [' '.join((item['question'] + " " + item['output'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]


print("MAUVE: ", mauve_score(predictions, references))

print("ROUGE: ", compute_rouge(normalized_data))

## [Match] PopQA, TriviaQA, PubHealth, ARC-C

In [1]:
import copy
import json
from metrics import *
from utils import *

# data = load_file("/shared/eng/pj20/firas_data/test_datasets/results/popqa_sonnet_retrieval_top_5.jsonl")
with open("/shared/eng/pj20/firas_data/test_datasets/triviaqa_test_output_sonnet_sonnet.json", "r") as f:
    data_ = json.load(f)

data = []
for i in range(len(data_['output'])):
    data.append({"output": data_['output'][i].lower(), "golds": [d.lower() for d in data_['answer'][i]]})


/home/pj20/miniconda3/envs/handbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pj20/miniconda3/envs/handbook/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from tqdm import tqdm

data_["metric_result"] = []
for i in tqdm(range(len(data))):
    data_["metric_result"].append(cosine_match(data[i]["output"], data[i]["golds"]))
    if data_["metric_result"][i] == 1:
        data_["answer"][i].append(data[i]["output"])

print("Match: ", np.mean(data_["metric_result"]))

100%|██████████| 999/999 [00:32<00:00, 30.91it/s] 

Match:  0.24024024024024024


In [19]:
np.mean(data_["metric_result"][:243])

0.757201646090535

In [18]:
i = 691
data_["metric_result"][i], data_["answer"][i],data_["output"][i]


(1, ['', '12', 'twelve', 'croatia'], 'Croatia')

In [ ]:
import copy
import json
from metrics import *
from utils import *
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

In [8]:
# Get embeddings for output and all gold answers
output_emb = model.encode(data_["output"][i], convert_to_tensor=True)
gold_embs = model.encode(data_["answer"][i], convert_to_tensor=True)
    
    # Calculate cosine similarities
similarities = F.cosine_similarity(output_emb.unsqueeze(0), gold_embs)
    

similarities


tensor([0.8377, 0.2373, 0.7109, 0.6616, 0.2947, 0.8449, 0.4855, 0.5121, 0.4533,
        0.0541, 0.6941, 0.8485, 0.1473, 0.4985, 1.0000, 0.3116, 0.7921, 0.3176,
        0.3021, 0.9029, 0.5923, 0.6585, 0.6905, 0.5659, 0.2425, 0.7421, 0.9029],
       device='cuda:0')

## [F1] 2Wiki

In [3]:
import copy
import json
from metrics import *
from utils import *

data = load_file("/shared/eng/pj20/firas_data/test_datasets/results/popqa_sonnet_retrieval_top_5.jsonl")

In [ ]:
for item in data:
    metric_result = f1_score(item["output"], item["golds"])
    item["metric_result"] = metric_result

print("F1: ", np.mean([item["metric_result"] for item in data]))

## [FS] Bio

python -m factscore.factscorer --data_path YOUR_OUTPUT_FILE  --model_name retrieval+ChatGPT --cache_dir YOUR_CACHE_DIR --openai_key YOUR_OPEN_AI_KEY --verbose